In [1]:
# installing kaggle library
! pip install kaggle

Upload Kaggle.json file

In [3]:
# configuring the path of kaggle.json file
import shutil
import os

# Create the .kaggle directory if it doesn't exist
kaggle_dir = os.path.expanduser('~/.kaggle')
if not os.path.exists(kaggle_dir):
    os.makedirs(kaggle_dir)

# Move the kaggle.json file to the .kaggle directory
shutil.move('kaggle.json', os.path.join(kaggle_dir, 'kaggle.json'))

# Set permission for the file (Windows doesn't use chmod, so this can be skipped)
# This is generally for Unix-based systems, so in Windows you can ignore it


'C:\\Users\\santh/.kaggle\\kaggle.json'

Verify Kaggle Configuration

In [4]:
# Test if Kaggle API is working
!kaggle datasets list

ref                                                          title                                               size  lastUpdated          downloadCount  voteCount  usabilityRating  
-----------------------------------------------------------  -------------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
valakhorasani/mobile-device-usage-and-user-behavior-dataset  Mobile Device Usage and User Behavior Dataset       11KB  2024-09-28 20:21:12           7306        132  1.0              
lainguyn123/student-performance-factors                      Student Performance Factors                         94KB  2024-09-02 10:53:57          35676        630  1.0              
valakhorasani/gym-members-exercise-dataset                   Gym Members Exercise Dataset                        22KB  2024-10-06 11:27:38           3680         76  1.0              
mohamedyosef101/2024-olympics-medals-and-economic-status     2024 Olympics Medal

Importing Twitter Sentiment Dataset

In [6]:
# API to fetch dataset from kaggle
! kaggle datasets download -d kazanova/sentiment140

Dataset URL: https://www.kaggle.com/datasets/kazanova/sentiment140
License(s): other
sentiment140.zip: Skipping, found more recently modified local copy (use --force to force download)


Unzip the Dataset

In [8]:
import zipfile
import os

# Unzip the dataset
with zipfile.ZipFile('sentiment140.zip', 'r') as zip_ref:
    zip_ref.extractall()  # Extracts to the current directory

# Extract to a specific folder
with zipfile.ZipFile('sentiment140.zip', 'r') as zip_ref:
    zip_ref.extractall('path/to/directory')  # Replace with your directory

Import the Dataset

In [9]:
import pandas as pd
import os

# Load the dataset
df = pd.read_csv('training.1600000.processed.noemoticon.csv', encoding='latin-1')  # Adjust the file name based on the extracted files
# List files in the current directory
os.listdir()


['.ipynb_checkpoints',
 'path',
 'sentiment140.zip',
 'training.1600000.processed.noemoticon.csv',
 'Untitled.ipynb']

Understanding the dataset

In [11]:
df.columns=['target','id','date','flag','user','text']
df.head()

,target,id,date,flag,user,text
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


Importing the dependencies

In [12]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [13]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\santh\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [14]:
# Printing the stopwords in english
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

Data Processing

loading the data from csv file to pandas dataframe

In [18]:
# Use raw string notation by adding 'r' before the string
twitter_data = pd.read_csv(r"C:\Users\santh\Untitled Folder\training.1600000.processed.noemoticon.csv", encoding='ISO-8859-1')
# Assign column names
twitter_data.columns = ['target', 'id', 'date', 'flag', 'user', 'text']

# Display the first few rows of the dataset
twitter_data.head()


,target,id,date,flag,user,text
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [20]:
# checkind the number of rows and columns
twitter_data.shape

(1599999, 6)

In [21]:
#counting the number of missing values

twitter_data.isnull().sum()

target    0
id        0
date      0
flag      0
user      0
text      0
dtype: int64

In [22]:
# checking the distribution of the target column

twitter_data['target'].value_counts()

target
4    800000
0    799999
Name: count, dtype: int64

Converting the target "4" to "1"

In [23]:
twitter_data.replace({'target': {4:1}}, inplace=True)

In [24]:
# checking the distribution of the target column

twitter_data['target'].value_counts()

target
1    800000
0    799999
Name: count, dtype: int64

0 --> Negative Tweet

1 --> Positive Tweet

Stemming

Stemming is the process of reducing a word to its Root word

exmple: actor, actress, acting = act

In [25]:
port_stem = PorterStemmer()

In [26]:
def steming(content):

  stemmed_content = re.sub('[^a-zA-z]',' ',content)
  stemmed_content = stemmed_content.lower()
  stemmed_content = stemmed_content.split()
  stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  stemmed_content = ' '.join(stemmed_content)


  return stemmed_content

In [29]:
twitter_data['stemmed_content'] = twitter_data['text'].apply(steming)

In [28]:
twitter_data.head()

,target,id,date,flag,user,text,stemmed_content
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,upset updat facebook text might cri result sch...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,kenichan dive mani time ball manag save rest g...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,whole bodi feel itchi like fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",nationwideclass behav mad see
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew,kwesidei whole crew


In [30]:
print(twitter_data['stemmed_content'])

0          upset updat facebook text might cri result sch...
1          kenichan dive mani time ball manag save rest g...
2                            whole bodi feel itchi like fire
3                              nationwideclass behav mad see
4                                        kwesidei whole crew
                                 ...                        
1599994                           woke school best feel ever
1599995    thewdb com cool hear old walt interview http b...
1599996                         readi mojo makeov ask detail
1599997    happi th birthday boo alll time tupac amaru sh...
1599998    happi charitytuesday thenspcc sparkschar speak...
Name: stemmed_content, Length: 1599999, dtype: object


In [31]:
print(twitter_data['target'])

0          0
1          0
2          0
3          0
4          0
          ..
1599994    1
1599995    1
1599996    1
1599997    1
1599998    1
Name: target, Length: 1599999, dtype: int64


In [32]:
#separating the data and label

X = twitter_data['stemmed_content'].values
Y = twitter_data['target'].values

In [33]:
print(X)

['upset updat facebook text might cri result school today also blah'
 'kenichan dive mani time ball manag save rest go bound'
 'whole bodi feel itchi like fire' ... 'readi mojo makeov ask detail'
 'happi th birthday boo alll time tupac amaru shakur'
 'happi charitytuesday thenspcc sparkschar speakinguph h']


In [34]:
print(Y)

[0 0 0 ... 1 1 1]


Splitting the data to training data and test data

In [35]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, stratify=Y, random_state=2)

print(X.shape, X_train.shape, X_test.shape)

(1599999,) (1279999,) (320000,)


In [36]:
print(X_train)

['watch saw iv drink lil wine'
 'hire anoth employe gourmet point current hurri'
 'punish know much work tomorrow everyon els get day' ...
 'ohjustjak awkward crush nph amazingg'
 'oliveandfig wow never tweet nah yet tuesday spoil'
 'that_girl_jenn stay twitter b c got back late amp tri relax enough sleep work amp awak']


In [37]:
print(X_test)

['school someon slap acrosss face caus stay class total dragggg poor poor kitti'
 'ah may show w ruth kim amp geoffrey sanhueza'
 'dad comput turn even though weird mean download song want' ...
 'destini nevertheless hooray member wonder safe trip'
 'strawberri heavi appar broke blender epic fuck fail' 'supersandro thank']


In [38]:
# converting the texttual data to numerical data

vectorizer = TfidfVectorizer()

X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

In [39]:
print(X_train)

  (0, 464907)	0.44755968424484005
  (0, 247566)	0.4211677185669695
  (0, 116313)	0.37428429359397636
  (0, 194742)	0.5293567683337778
  (0, 373033)	0.35766686591229396
  (0, 458362)	0.2721675481983611
  (1, 181532)	0.3783460265618032
  (1, 92615)	0.3300113641102886
  (1, 336609)	0.31277964698237826
  (1, 160607)	0.49260230408176026
  (1, 127172)	0.4307005603768187
  (1, 23001)	0.25281727079702565
  (1, 175881)	0.39754131995016506
  (2, 100219)	0.22341924630468818
  (2, 154375)	0.22130115521624924
  (2, 125253)	0.3986508849353605
  (2, 132426)	0.33174383935487023
  (2, 433486)	0.28960001302922206
  (2, 467565)	0.23562584799521835
  (2, 294403)	0.2849989296353406
  (2, 232688)	0.26260681919651707
  (2, 343356)	0.5852851415328681
  (3, 367795)	0.30871788531388655
  (3, 50489)	0.6886835448480676
  (3, 349108)	0.46555629423386596
  :	:
  (1279996, 311691)	0.47880627020252514
  (1279996, 16702)	0.4366940449213088
  (1279996, 32710)	0.37397170429854387
  (1279996, 91074)	0.3425177168682021
  

In [40]:
print(X_test)

  (0, 435028)	0.26230925717781944
  (0, 401776)	0.25392304109096725
  (0, 394815)	0.24640650445812856
  (0, 390406)	0.3972933108545502
  (0, 374361)	0.22348363749749403
  (0, 337530)	0.5234569549218435
  (0, 231083)	0.3307953247586049
  (0, 134819)	0.2732428887095021
  (0, 80775)	0.2653273474229123
  (0, 68745)	0.2640470757739768
  (1, 385159)	0.2423253397998145
  (1, 366122)	0.4811717662516511
  (1, 270662)	0.28298457606840066
  (1, 229266)	0.40540503595022426
  (1, 153746)	0.5748549434332728
  (1, 17977)	0.20793062743488835
  (1, 8813)	0.3026967059147128
  (2, 460305)	0.37056303922275935
  (2, 457786)	0.22643285141278216
  (2, 439972)	0.33256039758138023
  (2, 428010)	0.2705826510099419
  (2, 395096)	0.30217576065585394
  (2, 272647)	0.30725493909161034
  (2, 132131)	0.2729599284373498
  (2, 114774)	0.3685376425854966
  :	:
  (319995, 323457)	0.3883302771684808
  (319995, 203834)	0.3857804933967842
  (319995, 187526)	0.30322986158885457
  (319995, 75382)	0.32077047552960325
  (319995

Training the ML Model

Logistic Regression

In [ ]:
model = LogisticRegression(max_iter = 1000)

In [42]:
model.fit(X_train, Y_train)

LogisticRegression(max_iter=1000)

Model Evaluation

Accuracy Score

In [43]:
# accuracy score on the trainning data

X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(Y_train, X_train_prediction)

In [44]:
print('Accuracy score on the training data:', training_data_accuracy)

Accuracy score on the training data: 0.8117670404430003


In [51]:
# accuracy score on the test data

X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(Y_test, X_test_prediction)

In [52]:
print('Accuracy score on the test data:', test_data_accuracy)

Accuracy score on the test data: 0.7785375


Model accuracy =77.8%

Saving the trained model

In [46]:
import pickle

In [54]:
filename = 'trained_model.sav'
pickle.dump(model, open(filename, 'wb'))

Using the saved model for future prediction

In [55]:
# Loading the saved model
loaded_model = pickle.load(open('C:\\Users\\santh\\Untitled Folder\\trained_model.sav', 'rb'))

In [56]:
X_new = X_test[200]
print(Y_test[200])

prediction = model.predict(X_new)
print(prediction)


if (prediction[0] == 0):
  print('Negative Tweet')

else:
  print('Positive Tweet')

1
[1]
Positive Tweet


In [60]:
X_new = X_test[3]
print(Y_test[3])

prediction = model.predict(X_new)
print(prediction)


if (prediction[0] == 0):
  print('Negative Tweet')

else:
  print('Positive Tweet')

0
[1]
Positive Tweet
